# [LangChain 怎麼玩？ LCEL (LangChain Expression Language) 篇，一定要認識的 LangChain 核心](https://myapollo.com.tw/blog/langchain-expression-language/#google_vignette)

## LCEL 有 2 個重要的目的。

### 統一的協定(Protocol) 
1. A unified interface: Every LCEL object implements the Runnable interface, which defines a common set of invocation methods (invoke, batch, stream, ainvoke, …). This makes it possible for chains of LCEL objects to also automatically support these invocations. That is, every chain of LCEL objects is itself an LCEL object.

LangChain 制定的協定(protocol)，每個元件例如(Prompt, ChatModel, LLM, OutputParser, Retriever, Tool 這些都是 LangChain 基本元件)都必須實作 1 個稱為 “Runnable” 的協定，這個協定至少包含以下方法（這些方法很重要，如果要將 Chain 應用變成 API 開放給他人使用，也可能需要實作以下方法）：

* invoke (支援單一輸入、單一輸出)
* batch (支援多個輸入、多個輸出)
* stream (支援有部分結果就輸出的模式)
* ainvoke (async 版本的 invoke)
* abatch (async 版本的 batch)
* astream (async 版本的 stream)

### 提供組合原型(Composition primitives)，讓 Chain 的開發變得簡易 
2. Composition primitives: LCEL provides a number of primitives that make it easy to compose chains, parallelize components, add fallbacks, dynamically configure chain internal, and more.

LCEL 提供元件組合的功能，也就是本系列文章中經常使用到的 | 運算子，例如：
```
prompt | llm
```
組合出來的 Chain 原型(primitives)分為 2 種：

* RunnableSequence
* RunnableParallel

## RunnableSequence #
RunnableSequence 會依序執行每個 Runnable , 例如 prompt | llm | output 裡的 prompt, llm, output 都是 Runnable , 它們透過 | 運算子串起來之後，就是 RunnableSequence ，執行上是依序執行，例如下列程式碼範例，可以體驗到何謂 RunnableSequence :



In [3]:
# from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

prompt_output = prompt.invoke({"input": 'Hi there'})
llm_output = llm.invoke(prompt_output)
answer = StrOutputParser().invoke(llm_output)

print(llm_output)
print("=======================")
print(answer)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None} id='run-68b3275b-c937-409c-9641-083bb691b1da-0' usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18}
Hello! How can I assist you today?


In [4]:
chain = prompt | llm
res = chain.invoke({"input": 'Hi there'})
print(res)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None} id='run-bdf4df31-63ec-4e24-b178-98ae2912d7a6-0' usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18}


## RunnableParallel #
RunnableParallel 則是可以同時執行 Runnable, 它的範例可以用 LangChain 的 RunnableLambda 體驗：

In [5]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def add_two(x: int) -> int:
    return x + 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)

parallel = {"runnable_1": runnable_1, "runnable_2": runnable_2}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

{'runnable_1': 2, 'runnable_2': 3}


In [6]:
parallel = {"r1": runnable_1, "r2": runnable_2}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

{'r1': 2, 'r2': 3}


## RunnableBranch 
目前為止，我們已經接觸 `Runnable`, `RunnableSequence`, `RunnableParallel`, `RunnableLambda`, 還有個變化型稱為 `RunnableBranch`。

`RunnableBranch` 可以讓我們動態決定要走哪個分支執行下 1 個 Runnable, 例如你可以透過使用者不同的輸入，而採用不同的語言模型，例如：



In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

default_prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

python_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a full-stack developer"),
    ("user", "python {input}"),
])

default_chain = default_prompt | llm # you can customize the chain here
python_chain = python_prompt | llm  # you can customize the chain here

def route(x):
    if 'python' in x['input']:
        return python_chain
    return default_chain

chain = RunnableLambda(route)

print(type(chain))

# print(chain.invoke({"input": "python is the best"}))

llm_output = chain.invoke({"input": "python is the best"})

res = StrOutputParser().invoke(llm_output)

print(res)


<class 'langchain_core.runnables.base.RunnableLambda'>
Python is indeed considered one of the best programming languages by many developers for several reasons:

1. **Readability**: Python has a clean and easy-to-read syntax, which makes it accessible for beginners and allows experienced developers to write code more efficiently.

2. **Versatility**: Python can be used for a wide range of applications, including web development, data analysis, artificial intelligence, scientific computing, automation, and more.

3. **Strong Community and Libraries**: Python has a large and active community, which means there are numerous libraries and frameworks available for various tasks (like Django and Flask for web development, Pandas and NumPy for data analysis, and TensorFlow and PyTorch for machine learning).

4. **Cross-Platform Compatibility**: Python runs on various platforms, including Windows, macOS, and Linux, making it a flexible option for development.

5. **Rapid Development**: The sim

### Example from LangChain
ref: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.branch.RunnableBranch.html

In [13]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), lambda x: x + 1),
    (lambda x: isinstance(x, float), lambda x: x * 2),
    lambda x: "goodbye",
)

print(branch.invoke("hello")) # "HELLO"
print(branch.invoke(None)) # "goodbye"
print(branch.invoke(1)) # 2

HELLO
goodbye
2


## RunnablePassthrough 
```
Runnable to passthrough inputs unchanged or with additional keys.
```
如果你想對輸入做些加工的話，或者對前者 Runnable 輸出的結果做修改的話，例如修改使用者 prompt 加入關鍵詞，則可以使用 RunnablePassthrough 。

下列是修改使用者 prompt 的範例：

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = RunnablePassthrough.assign(input=lambda x: x['input'] + ' this is important to me.') | prompt
print(chain.invoke({"input": "python is the best."}))

messages=[HumanMessage(content='python is the best. this is important to me.')]


## 輸入/輸出 (Input & Output Schema) 
從各個範例可以得知每個 Runnable 都有輸入與輸出，可是要怎麼知道它到底需要輸入什麼結構的資料，以及它會輸出什麼結構的資料呢？

這時可以查看 Runnable 的輸入與輸出的 schema, 這些 schema 被存在 `input_schema` 與 `output_schema` 兩個屬性中，只要是 Runnable 都可以查看其輸入與輸出結構。

### Example 1:

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

In [17]:
print(prompt.input_schema)
print(prompt.input_schema.schema())

<class 'pydantic.v1.main.PromptInput'>
{'title': 'PromptInput', 'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']}


In [18]:
print(prompt.output_schema)
print(prompt.output_schema.schema())

<class 'pydantic.v1.main.ChatPromptTemplateOutput'>
{'title': 'ChatPromptTemplateOutput', 'anyOf': [{'$ref': '#/definitions/StringPromptValue'}, {'$ref': '#/definitions/ChatPromptValueConcrete'}], 'definitions': {'StringPromptValue': {'title': 'StringPromptValue', 'description': 'String prompt value.', 'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}, 'type': {'title': 'Type', 'default': 'StringPromptValue', 'enum': ['StringPromptValue'], 'type': 'string'}}, 'required': ['text']}, 'ToolCall': {'title': 'ToolCall', 'type': 'object', 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'args': {'title': 'Args', 'type': 'object'}, 'id': {'title': 'Id', 'type': 'string'}, 'type': {'title': 'Type', 'enum': ['tool_call'], 'type': 'string'}}, 'required': ['name', 'args', 'id']}, 'InvalidToolCall': {'title': 'InvalidToolCall', 'type': 'object', 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'args': {'title': 'Args', 'type': 'string'}, 'id': {'tit

### Example 2

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages([
    SystemMessage("you are a full-stack developer"),
    # MessagesPlaceholder(variable_name='chat_history'),
    HumanMessage("python {input1}"),
])

In [32]:
# print(prompt.input_schema)
print(prompt.input_schema.schema())
print(prompt.output_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {}}
{'title': 'ChatPromptTemplateOutput', 'anyOf': [{'$ref': '#/definitions/StringPromptValue'}, {'$ref': '#/definitions/ChatPromptValueConcrete'}], 'definitions': {'StringPromptValue': {'title': 'StringPromptValue', 'description': 'String prompt value.', 'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}, 'type': {'title': 'Type', 'default': 'StringPromptValue', 'enum': ['StringPromptValue'], 'type': 'string'}}, 'required': ['text']}, 'ToolCall': {'title': 'ToolCall', 'type': 'object', 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'args': {'title': 'Args', 'type': 'object'}, 'id': {'title': 'Id', 'type': 'string'}, 'type': {'title': 'Type', 'enum': ['tool_call'], 'type': 'string'}}, 'required': ['name', 'args', 'id']}, 'InvalidToolCall': {'title': 'InvalidToolCall', 'type': 'object', 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'args': {'title': 'Args', 'type': 'string'}, 'i

In [33]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model='llama2')
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
print(chain.input_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']}


## 列印 Chain 的樣子 
除了 input / output schema 的資訊， LangChain 也有提供 1 個方便的函式可以查看 chain 的長相，該方法為 <chain>.get_graph().print_ascii() 。

舉下列範例為例：

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
chain.get_graph().print_ascii()

    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
    +------------+     
    | ChatOpenAI |     
    +------------+     
           *           
           *           
           *           
 +------------------+  
 | ChatOpenAIOutput |  
 +------------------+  
